In [232]:
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats, leaguegamefinder, commonteamroster, leagueseasonmatchups
from nba_api.stats.static import players, teams
from os import path
import pandas as pd
import pickle
import numpy as np

In [2]:
""" for saving and loading things to a pickle file. Don't need to add extension to the file name """
def save_obj(obj, filename, dirname='pickle_files', ):
    with open(path.join(dirname, filename + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(filename, dirname='pickle_files'):
    with open(path.join(dirname, filename + '.pkl'), 'rb') as f:
        return pickle.load(f)

In [97]:
teams_df = pd.DataFrame(teams.get_teams())
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [4]:
team_ids = teams_df.loc[:, ("id","abbreviation")]

In [62]:
team_ids["abbreviation"]

0     ATL
1     BOS
2     CLE
3     NOP
4     CHI
5     DAL
6     DEN
7     GSW
8     HOU
9     LAC
10    LAL
11    MIA
12    MIL
13    MIN
14    BKN
15    NYK
16    ORL
17    IND
18    PHI
19    PHX
20    POR
21    SAC
22    SAS
23    OKC
24    TOR
25    UTA
26    MEM
27    WAS
28    DET
29    CHA
Name: abbreviation, dtype: object

In [64]:
# *** should only have to run the loading cell below ***
# get team info into dataframes
# store dictionary key = team abbreviation, value = df with game data
import time

team_names = {}
for i in range(len(team_ids)): 
    temp_id = team_ids["id"][i]
    temp_team_name = team_ids["abbreviation"][i]
    print('getting team: {0}...'.format(temp_team_name))
    team_names[temp_team_name] = leaguegamefinder.LeagueGameFinder(team_id_nullable=temp_id).get_data_frames()[0]
    time.sleep(1)

getting team: ATL...


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [6]:
""" save dictionary to a file called team_names under pickle_files folder"""
# save_obj(team_names, 'team_names')

NameError: name 'team_names' is not defined

In [73]:
""" load the team_names dictionary in pickle_files """
team_names = load_obj('team_names')

In [234]:
team_names_test = {}

ids = team_ids["id"]
abr = team_ids["abbreviation"]

for i in range(len(ids)):
    real_value = team_names[abr[i]]
    test_key = ids[i]
    team_names_test[test_key] = real_value

In [172]:
# for game in team_names['CHA'].values.tolist():
#     if (game[2] != "CHA"):
#         print(game[2])

all_abrs = {}
for team in team_names.keys():
    df = team_names[team]
    abrs = set()
    for game in df.values.tolist():
        abrs.add(game[2])
    for abr in abrs:
        all_abrs[abr] = df
print(len(all_abrs))
all_abrs['GOS']

42


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612744,GSW,Golden State Warriors,0021900107,2019-11-06,GSW @ HOU,None,0,8,...,NaN,0.0,3.0,3.0,3,0,0,0,0,1.0
1,22019,1610612744,GSW,Golden State Warriors,0021900096,2019-11-04,GSW vs. POR,W,241,127,...,0.917,8.0,39.0,47.0,28,4,4,10,17,9.0
2,22019,1610612744,GSW,Golden State Warriors,0021900083,2019-11-02,GSW vs. CHA,L,241,87,...,0.769,5.0,35.0,40.0,23,5,5,18,22,-6.0
3,22019,1610612744,GSW,Golden State Warriors,0021900076,2019-11-01,GSW vs. SAS,L,240,110,...,0.905,7.0,32.0,39.0,25,4,2,11,22,-17.0
4,22019,1610612744,GSW,Golden State Warriors,0021900065,2019-10-30,GSW vs. PHX,L,240,110,...,0.833,7.0,36.0,43.0,24,8,6,18,22,-11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3246,21983,1610612744,GOS,Golden State Warriors,0028300048,1983-11-05,GOS vs. NYK,W,240,102,...,0.711,25.0,18.0,43.0,20,16,4,17,27,NaN
3247,21983,1610612744,GOS,Golden State Warriors,0028300040,1983-11-03,GOS @ UTH,L,240,112,...,0.880,27.0,26.0,53.0,22,11,4,19,31,NaN
3248,21983,1610612744,GOS,Golden State Warriors,0028300029,1983-11-01,GOS @ KCK,L,240,114,...,0.750,12.0,18.0,30.0,29,15,3,16,28,NaN
3249,21983,1610612744,GOS,Golden State Warriors,0028300013,1983-10-29,GOS vs. POR,W,240,97,...,0.744,12.0,31.0,43.0,21,9,1,16,37,NaN


In [173]:
""" gets the season games given the seasonID and abbreviation of the team """
""" Season is based on the year that the season STARTED in. I.E 2016-2017 has seasonID 2016"""
""" Reg season starts with 2, playoffs 4, pre 1 """
""" 
For some reason summer league and preseason games are sometimes included in reg season IDs.
The function accounts for this by only looking at Game IDs that start with 0021 for reg reason,
0041 for playoffs, 0011 for preseason
"""

def get_season_team(seasonID, team_id, team_names):
    seasonID = str(seasonID)
    gameID_prefix = seasonID[0]
    return team_names[team_id].loc[(team_names[team_id]["SEASON_ID"].str.contains(seasonID)) & \
                                    (team_names[team_id]['GAME_ID'].str.startswith('00{0}'.format(gameID_prefix)))]


""" gets the number of rows in the passed df"""
def num_rows(df):
    return len(df.index)

""" returns the number of features of the df"""
def num_features(df):
    return len(df.columns)

""" returns a list of the features from the start index moving forward. Using 9 for now """
def get_features(df, start_idx=9):
    return [feature for feature in df.columns.tolist()[start_idx:]]

""" 
util func: gets last n games given the curr game
in the df with the correct season, year
"""
def last_n_games(df, n, curr_game):
    n_rows = num_rows(df)
    # the first row is the latest game. I.E for reg season it's 82nd game
    
"""
returns avg team stats vector over the last n games given
the current game. Assumes games 0 indexed
"""
def get_team_avgs(season_games, curr_game, n):
    total_games = num_rows(season_games)
    assert curr_game > n, "curr game ({0}) > n ({1})".format(curr_game, n)
    assert total_games > n, "n ({0}) is more than the total number of games ({1})".format(n, total_games)
    # the real index of the current game
    game_idx = total_games - curr_game - 1
    # start at 9th index to get meaningful stats
    start_idx = 9
    team_avgs = np.zeros(num_features(season_games) - start_idx)
    for i in range(game_idx - n, game_idx):
        game_stats = np.array(season_games.iloc[i].tolist()[start_idx:])
        team_avgs += game_stats
    return team_avgs / n

def print_team_avgs(season_games, team_avgs):
    features = get_features(season_games)
    for stat, feature in zip(team_avgs, features):
        print("{0}:{1}".format(feature, stat))

In [174]:
""" 
get the 2018 season games for the Atlanta Hawks. 
"""
atl_reg_season = get_season_team(22018, 1610612737, team_names_test)
"""
print the avgs of the 10 games before the 16th game that the Hawks played in the 2017-2018 season
"""
atl_team_avgs = get_team_avgs(atl_reg_season, 16, 10)
print_team_avgs(atl_reg_season, atl_team_avgs)

PTS:108.7
FGM:40.8
FGA:90.5
FG_PCT:0.45099999999999996
FG3M:10.9
FG3A:36.0
FG3_PCT:0.3
FTM:16.2
FTA:21.7
FT_PCT:0.7397000000000002
OREB:10.5
DREB:32.5
REB:43.0
AST:25.3
STL:10.2
BLK:6.2
TOV:18.2
PF:23.2
PLUS_MINUS:-9.0


In [156]:
team_names["ATL"].loc[team_names["ATL"]["SEASON_ID"].str.contains("21999")]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1795,21999,1610612737,ATL,Atlanta Hawks,0029901181,2000-04-19,ATL @ IND,L,240,92,...,0.690,10,31,41,18,12.0,5,6,17,-19.0
1796,21999,1610612737,ATL,Atlanta Hawks,0029901168,2000-04-18,ATL vs. NJN,W,292,111,...,0.792,17,43,60,21,9.0,10,21,25,2.0
1797,21999,1610612737,ATL,Atlanta Hawks,0029901153,2000-04-16,ATL vs. CLE,W,239,104,...,0.645,12,35,47,25,6.0,3,20,25,3.0
1798,21999,1610612737,ATL,Atlanta Hawks,0029901138,2000-04-14,ATL vs. PHI,L,240,92,...,0.938,14,25,39,19,9.0,4,14,24,-12.0
1799,21999,1610612737,ATL,Atlanta Hawks,0029901124,2000-04-12,ATL @ CHH,L,239,87,...,0.778,10,21,31,26,7.0,5,17,26,-32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872,21999,1610612737,ATL,Atlanta Hawks,0029900067,1999-11-10,ATL @ VAN,L,264,97,...,0.750,10,31,41,17,9.0,7,14,18,-5.0
1873,21999,1610612737,ATL,Atlanta Hawks,0029900053,1999-11-08,ATL @ DEN,L,241,100,...,0.733,13,26,39,21,2.0,6,12,22,-15.0
1874,21999,1610612737,ATL,Atlanta Hawks,0029900037,1999-11-06,ATL vs. CHI,W,241,113,...,0.647,13,29,42,21,10.0,3,11,30,16.0
1875,21999,1610612737,ATL,Atlanta Hawks,0029900021,1999-11-04,ATL vs. MIL,L,239,109,...,0.700,17,29,46,22,3.0,5,26,26,-10.0


In [157]:
team_names_test[1610612737].loc[team_names_test[1610612737]["SEASON_ID"].str.contains("2018")]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
18,22018,1610612737,ATL,Atlanta Hawks,0021801220,2019-04-10,ATL vs. IND,L,240,134,...,0.816,22,39,61,29,5.0,7,17,25,-1.0
19,22018,1610612737,ATL,Atlanta Hawks,0021801202,2019-04-07,ATL @ MIL,L,240,107,...,0.526,9,39,48,25,2.0,3,11,28,-8.0
20,22018,1610612737,ATL,Atlanta Hawks,0021801181,2019-04-05,ATL @ ORL,L,240,113,...,0.677,10,28,38,21,16.0,4,14,21,-36.0
21,22018,1610612737,ATL,Atlanta Hawks,0021801168,2019-04-03,ATL vs. PHI,W,240,130,...,0.786,11,33,44,29,7.0,7,11,26,8.0
22,22018,1610612737,ATL,Atlanta Hawks,0021801162,2019-04-02,ATL @ SAS,L,240,111,...,0.650,11,32,43,26,13.0,2,11,18,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,22018,1610612737,ATL,Atlanta Hawks,1521800024,2018-07-08,ATL vs. POR,L,198,68,...,0.615,9,34,43,18,11.0,7,15,17,-17.0
110,22018,1610612737,ATL,Atlanta Hawks,1521800014,2018-07-07,ATL vs. NYK,L,200,89,...,0.591,16,35,51,22,7.0,5,16,29,-6.0
111,22018,1610612737,ATL,Atlanta Hawks,1621800006,2018-07-05,ATL @ UTA,L,201,87,...,0.654,15,33,48,16,11.0,5,17,25,2.2
112,22018,1610612737,ATL,Atlanta Hawks,1621800003,2018-07-03,ATL vs. SAS,L,200,81,...,0.545,14,29,43,12,9.0,2,12,25,-26.2


In [176]:
def find_opponent_id(opp_abr, season_id):
    
    return all_abrs[opp_abr].loc[all_abrs[opp_abr]["SEASON_ID"].str.contains(str(season_id))]["TEAM_ID"].iloc[0]

In [177]:
def find_opponent(n, curr_team, season_id):
    full_df = get_season_team(season_id, curr_team, team_names_test).reset_index()
    
    df_of_game = full_df.iloc[[-n]]
    opp_abr = df_of_game["MATCHUP"].values[0][-3::]
    return find_opponent_id(opp_abr, season_id)

In [179]:
"""get the opponent of the Atlanta Hawks's 2nd game of the the 2018 season"""

find_opponent(2, 1610612737, 22018)

1610612763

In [253]:
def convert_to_01s(letter):
    return 1 if letter == "W" else 0

In [264]:
"""
return the difference vector
"""
def find_diff_vector(curr_game, n, curr_team, season_id):
    #curr_team_season is a df
    curr_team_season = get_season_team(season_id, curr_team, team_names_test)
    curr_team_avgs = get_team_avgs(curr_team_season, curr_game, n)
    
    # find win/loss for single curr_game
    outcome = curr_team_season["WL"].iloc[num_rows(curr_team_season) - curr_game - 1]
    outcome = convert_to_01s(outcome)
    
    opponent = find_opponent(curr_game, curr_team, season_id)
    opponent_team_season = get_season_team(season_id, opponent, team_names_test)
    opponent_team_avgs = get_team_avgs(opponent_team_season, curr_game, n)
    
    diff = np.subtract(curr_team_avgs, opponent_team_avgs)
   
    return diff, outcome

In [265]:
"""diff vector between Atlanta Hawks' stats and its opponents' stats on the Hawks' 16th game
based on the averages of the last 15 games before the nth game"""
find_diff_vector(16, 15, 1610612737, 22009)

(array([ 9.13333333,  3.73333333,  2.33333333,  0.02993333,  1.93333333,
         3.46666667,  0.06026667, -0.26666667,  1.26666667, -0.05326667,
        -1.73333333,  1.06666667, -0.66666667,  3.8       ,  0.66666667,
        -0.33333333, -2.46666667, -0.73333333, 12.        ]), 0)

In [216]:
# gets diff vectors for all the games 2-82 for given team and season
def get_season_diff_vectors(curr_team, season_id):
    diff_vector = []
    df_len = num_rows(get_season_team(season_id, curr_team, team_names_test))
    for i in range(2, df_len):
        diff_vector.append(find_diff_vector(i, i-1, curr_team, season_id))
    return diff_vector

In [217]:
get_season_diff_vectors(1610612737, 22011)

[array([ -2.   ,   3.   ,   2.   ,   0.028,   2.   ,   0.   ,   0.125,
        -10.   ,  -7.   ,  -0.245,  -4.   ,   5.   ,   1.   ,   1.   ,
         -5.   ,  -2.   ,  -2.   ,  -7.   ,  -3.   ]),
 array([-1.    ,  3.5   ,  1.    ,  0.0375, -0.5   , -9.5   ,  0.131 ,
        -7.5   , -4.5   , -0.174 , -4.    , 10.    ,  6.    ,  3.5   ,
        -1.    , -3.5   , -4.5   ,  3.    , 18.    ]),
 array([ 2.33333333, -1.        ,  9.33333333, -0.06166667,  0.66666667,
         4.66666667, -0.04533333,  3.66666667,  9.33333333, -0.14033333,
         3.33333333,  5.66666667,  9.        ,  4.33333333,  3.33333333,
         0.66666667,  0.33333333, -2.33333333, 14.33333333]),
 array([-3.750e+00,  7.500e-01,  1.750e+00,  1.000e-03,  2.000e+00,
         0.000e+00,  1.550e-01, -7.250e+00, -7.000e+00, -3.725e-02,
        -2.500e-01, -7.250e+00, -7.500e+00,  4.000e+00, -5.000e-01,
        -2.500e-01, -6.000e+00, -8.750e+00, -7.250e+00]),
 array([ 8.    ,  3.8   ,  2.6   ,  0.03  ,  1.    ,  1.8   ,  

In [192]:
# get difference vectors for all teams for one season

def diff_vectors_per_season(season_id):
    team_per_season_dict = {}
    for team in team_ids["id"]:
        team_per_season_dict[team] = get_season_diff_vectors(team, season_id)
    
    return team_per_season_dict

In [219]:
# get diff vectors for all seasons for all teams

def ultimate_diff_vectors():
    ultimate_dict = {}
    seasons = list(range(22009, 22019))

    for s in seasons:
        print(s)
        ultimate_dict[s] = diff_vectors_per_season(s)
        
    return ultimate_dict

In [220]:
"""
key = season
value = dictionary with key as team id, values as difference numpy arrays
"""
ultimate_dict = ultimate_diff_vectors()

22009
22010
22011
22012
22013
22014
22015
22016
22017
22018


In [231]:
# save ultimate_dict into a pickle file
#save_obj(ultimate_dict, 'ultimate_dict')
ultimate_dict_pickled = load_obj('ultimate_dict')
ultimate_dict_pickled[22018]

{1610612737: [array([19.   ,  1.   , -3.   ,  0.024,  5.   ,  1.   ,  0.125, 12.   ,
         18.   , -0.059, -3.   ,  8.   ,  5.   , 10.   , -1.   ,  0.   ,
          0.   ,  1.   , 12.   ]),
  array([-16.5   ,  -9.5   ,   1.5   ,  -0.1165,   2.5   ,  14.    ,
          -0.0485,   0.    ,   3.5   ,  -0.1415,  -1.    ,  14.    ,
          13.    ,   0.5   ,  -0.5   ,   4.    ,   5.5   ,  -7.    ,
          -7.    ]),
  array([-1.33333333e+00, -1.00000000e+00, -1.66666667e+00,  3.33333333e-04,
         -3.33333333e-01,  4.00000000e+00, -5.33333333e-02,  1.00000000e+00,
          3.66666667e+00, -7.10000000e-02, -1.33333333e+00,  3.33333333e-01,
         -1.00000000e+00,  2.66666667e+00,  6.66666667e-01,  2.00000000e+00,
         -3.33333333e-01,  3.66666667e+00, -1.96666667e+01]),
  array([ 7.5   ,  4.5   , -1.    ,  0.0615, -1.    ,  1.25  , -0.043 ,
         -0.5   ,  1.    , -0.077 , -1.5   , -1.25  , -2.75  ,  1.5   ,
          2.    ,  2.5   ,  6.75  ,  4.    , -5.75  ]),
  array([

# Find Labels
1. for season id 
2. for every team id 
3. make array looping through the last row of the frame to get the wins/losses

run through code for building ultimate dict to find ultimate wins/losses

In [235]:
team_names_test[1610612737].loc[team_names_test[1610612737]["SEASON_ID"].str.contains("22018")]["WL"].iloc[0]

'L'

In [229]:
all_abrs

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612737,ATL,Atlanta Hawks,0021900105,2019-11-06,ATL vs. CHI,None,24,8,...,1.000,1,3,4,1,0.0,0,3,0,-9.0
1,22019,1610612737,ATL,Atlanta Hawks,0021900099,2019-11-05,ATL vs. SAS,W,240,108,...,0.688,9,39,48,22,9.0,5,18,20,8.0
2,22019,1610612737,ATL,Atlanta Hawks,0021900066,2019-10-31,ATL vs. MIA,L,238,97,...,0.692,16,34,50,20,5.0,5,16,15,-9.0
3,22019,1610612737,ATL,Atlanta Hawks,0021900052,2019-10-29,ATL @ MIA,L,240,97,...,0.667,9,24,33,28,14.0,7,20,29,-15.0
4,22019,1610612737,ATL,Atlanta Hawks,0021900043,2019-10-28,ATL vs. PHI,L,240,103,...,0.688,8,37,45,23,12.0,3,21,25,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,21983,1610612737,ATL,Atlanta Hawks,0028300058,1983-11-06,ATL @ MIL,L,240,84,...,0.667,11,26,37,18,4.0,5,14,24,NaN
3248,21983,1610612737,ATL,Atlanta Hawks,0028300041,1983-11-04,ATL vs. CHI,W,240,103,...,0.692,19,27,46,31,14.0,13,18,27,NaN
3249,21983,1610612737,ATL,Atlanta Hawks,0028300027,1983-11-01,ATL vs. WAS,W,240,95,...,0.633,12,29,41,20,7.0,10,16,34,NaN
3250,21983,1610612737,ATL,Atlanta Hawks,0028300014,1983-10-29,ATL vs. DET,W,240,117,...,0.633,27,21,48,28,14.0,7,23,35,NaN


AttributeError: 'dict' object has no attribute 'equals'

In [11]:
team_names["ATL"].columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')